<h2 style="text-align: center;">An&aacute;lise Dataset Twitter</h2>
<h2 style="text-align: center;">Detec&ccedil;&atilde;o automatizada de Fake News e o problema da linguagem ofensiva e &oacute;dio</h2>

<p style="text-align: center;"><span style="color: #ff0000;"><strong>AVISO: Os dados utilizados, l&eacute;xicos e cadernos cont&ecirc;m conte&uacute;do racista, sexista, homof&oacute;bico e ofensivo de muitas outras maneiras.</strong></span></p>

***

> ```
@inproceedings{hateoffensive, 
  title={Automated Hate Speech Detection and the Problem of Offensive Language}, 
  author={Davidson, Thomas and Warmsley, Dana and Macy, Michael and Weber, Ingmar}, 
  booktitle={Proceedings of the 11th International AAAI Conference on Weblogs and Social Media}, 
  series={ICWSM '17}, 
  year={2017}, 
  location = {Montreal, Canada} 
  }
> ```

Repositório para Thomas Davidson, Dana Warmsley, Michael Macy e Ingmar Weber. 2017. "Detecção automática de fala de ódio e o problema da linguagem ofensiva". ICWSM. Você leu o artigo [aqui](https://aaai.org/ocs/index.php/ICWSM/ICWSM17/paper/view/15665) ou a nossa pré-impressão no [Arxiv](https://arxiv.org/abs/1703.04009) . Você pode encontrar mais informações na nossa [página do Github](https://github.com/t-davidson/hate-speech-and-offensive-language) .

***

### Guia de dados:

Os dados são armazenados como um CSV,contendo 5 colunas:

`count` = número de usuários do CrowdFlower que codificaram cada tweet (min é 3, algumas vezes mais usuários codificam um tweet quando os julgamentos não são confiáveis por CF).

`hate_speech` = número de usuários de CF que julgaram o tweet como discurso de ódio.

`offensive_language` = número de usuários de CF que julgaram o tweet ofensivo.

`neither` = número de utilizadores de CF que consideraram o tweet não ofensivo nem ofensivo.

`class`= rótulo de classe para a maioria dos usuários de CF. 0 - discurso de ódio, 1 - linguagem ofensiva, 2 - neutros

***

# Análise de Sentimentos usando Machine Learning

* Criando modelos para análise de sentimentos de tweets
* Teste com Modelo usando tag de negações
* Teste com Modelo usando Bigrams

In [1]:
from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

**Ler arquivo de dados e conta a quantidade de linhas**

In [2]:
dataset = pd.read_csv('labeled_data.csv', encoding='utf-8', sep=",")

**Exibe as 10 primeiras lihas de tweets**

In [3]:
dataset.tweet.head(10)

0    !!! RT @mayasolovely: As a woman you shouldn't...
1    !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2    !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3    !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4    !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5    !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...
6    !!!!!!"@__BrighterDays: I can not just sit up ...
7    !!!!&#8220;@selfiequeenbri: cause I'm tired of...
8    " &amp; you might not get ya bitch back &amp; ...
9    " @rhythmixx_ :hobbies include: fighting Maria...
Name: tweet, dtype: object

**Conta a quantidade de linhas de tweets neutros, positivos e negativos**

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns


sns.factorplot('hate_speech',data=dataset, kind='count', aspect=2)
sns.factorplot('offensive_language',data=dataset, kind='count', aspect=2)
sns.factorplot('neither',data=dataset, kind='count', aspect=2)

C:\Users\bruno\Anaconda3\lib\site-packages\seaborn\categorical.py:3666: UserWarning: The `factorplot` function has been renamed to `catplot`. The original name will be removed in a future release. Please update your code. Note that the default `kind` in `factorplot` (`'point'`) has changed `'strip'` in `catplot`.
  warnings.warn(msg)
C:\Users\bruno\Anaconda3\lib\site-packages\seaborn\categorical.py:3666: UserWarning: The `factorplot` function has been renamed to `catplot`. The original name will be removed in a future release. Please update your code. Note that the default `kind` in `factorplot` (`'point'`) has changed `'strip'` in `catplot`.
  warnings.warn(msg)
C:\Users\bruno\Anaconda3\lib\site-packages\seaborn\categorical.py:3666: UserWarning: The `factorplot` function has been renamed to `catplot`. The original name will be removed in a future release. Please update your code. Note that the default `kind` in `factorplot` (`'point'`) has changed `'strip'` in `catplot`.
  warnings.wa

In [5]:
dataset.count()

Unnamed: 0            24783
count                 24783
hate_speech           24783
offensive_language    24783
neither               24783
class                 24783
tweet                 24783
dtype: int64

## Pre-Processamento dos Dados

* Remove linhas duplicadas na base de dados
    - Problema na coleta dos dados.
* Remove Stopwords
* Faz Stemming nos dados
* Remove caracteres indesejados como links, pontuação etc.

In [6]:
dataset.drop_duplicates(['tweet'], inplace=True)

In [7]:
dataset.tweet.count()

24783

## ** Separando tweets e suas Classes**

In [8]:
tweets = dataset['tweet']
classes = dataset['class']

** Instala bibliotecas e baixa a base de dados**

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

**Funções de Pre-processamento de dados**

In [10]:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [11]:
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

In [12]:
def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

** Entenda como funciona cada função**

In [13]:
RemoveStopWords('fuck no that bitch dont even suck dick &#128514')

'fuck bitch dont even suck dick &#128514'

In [14]:
Stemming('@metroadlib: colored contacts in your eyes?')

'@metroadlib: colored contact in your eyes?'

In [15]:
Limpeza_dados('@oldpicsarchive: H.G.Wells and Charlie Chaplin http://t.co/sTiDDHK3WT" #History #Photography')

'@oldpicsarchive hgwells and charlie chaplin  #history #photography'

** Aplica as 3 funções de Pre-processamento nos dados**

In [16]:
def Preprocessing(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [stemmer.stem(i) for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Aplica a função em todos os dados:
tweets = [Preprocessing(i) for i in tweets]

In [17]:
Preprocessing('@oldpicsarchive: H.G.Wells and Charlie Chaplin http://t.co/sTiDDHK3WT" #History #Photography')

'@oldpicsarchiv hgwell charli chaplin #history #photography'

**Visualize os dados e veja como ficou após o pré-processamento**

In [18]:
tweets[:50]

['!!! rt @mayasolovely woman complain cleaning hous &amp man alway tak trash',
 '!!!!! rt @mleew17 boy dat coldtyg dwn bad cuffin dat hoe 1st place!!',
 '!!!!!!! rt @urkindofbrand dawg!!!! rt @80sbaby4lif ev fuck bitch start cry? confused shit',
 '!!!!!!!!! rt @c_g_anderson @viva_based look lik tranny',
 '!!!!!!!!!!!!! rt @shenikarobert shit he might tru might fak bitch told ya &#57361',
 '!!!!!!!!!!!!!!!!!!"@t_madison_x shit blow meclaim faithful somebody still fucking hoes! &#128514&#128514&#128514"',
 '!!!!!!"@__brighterday sit hat anoth bitch got much shit going on!"',
 "!!!!&#8220@selfiequeenbr caus i'm tired big bitch coming us skinny girls!!&#8221",
 '" &amp might get ya bitch back &amp that "',
 '" @rhythmixx_ hobbi includ fighting mariam" bitch',
 '" keek bitch curv everyon " lol walked conversation lik smh',
 '" murd gang bitch gang land "',
 '" hoe smok los ? " yea go ig',
 '" bad bitch thing lik "',
 '" bitch get "',
 '" bitch nigg mis "',
 '" bitch plz whatev "',
 '" bitch

## Criando o modelo

**Instancia o objeto que faz a vetorização dos dados de texto**

In [19]:
vectorizer = CountVectorizer(analyzer="word")

**Aplica o vetorizador nos dados de texto**

In [20]:
freq_tweets = vectorizer.fit_transform(tweets)
type(freq_tweets)

scipy.sparse.csr.csr_matrix

In [21]:
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

**Formato (Linhas, Colunas) da matriz**

In [22]:
freq_tweets.shape

(24783, 32699)

**Matriz**

In [23]:
freq_tweets.A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

** Testando o modelo com algumas instâncias simples**

In [24]:
# defina instâncias de teste dentro de uma lista
testes = ['Im elated i passed on my creative gene to my daughter but she also got the retarded gene too.',
          'Love having subs cause when my',
          'Steve McNair got murdered by his side bitch man life is crazy',
          'Unintended consequences of the current fed regulations.',
          'Lakers are a purple and yellow garbage can']

**Aplica a função de Pré-processamento nos dados**

In [25]:
testes = [Preprocessing(i) for i in testes]

In [26]:
# Transforma os dados de teste em vetores de palavras.
freq_testes = vectorizer.transform(testes)

In [27]:
# Fazendo a classificação com o modelo treinado.
for t, c in zip (testes,modelo.predict(freq_testes)):
    print (t +", "+ c)

TypeError: must be str, not numpy.int64

In [28]:
# Probabilidades de cada classe
print (modelo.classes_)
modelo.predict_proba(freq_testes).round(2)

[0 1 2]


array([[0.01, 0.99, 0.  ],
       [0.  , 0.98, 0.02],
       [0.  , 1.  , 0.  ],
       [0.02, 0.01, 0.97],
       [0.  , 0.02, 0.97]])

## ** Função de Tags de Negações**

* Acrescenta uma tag _NEG encontrada após um 'não'.
* Objetivo é dar mais peso para o modelo identificar uma inversão de sentimento da frase.
* Exemplos: 
    - Eu gosto de cachorros, positivo.
    - Eu **não** gosto de cachorros, negativo.

In [29]:
def marque_negacao(texto):
    negacoes = ['dont','not']
    negacao_detectada = False
    resultado = []
    palavras = texto.split()
    for p in palavras:
        p = p.lower()
        if negacao_detectada == True:
            p = p + '_NEG'
        if p in negacoes:
            negacao_detectada = True
        resultado.append(p)
    return (" ".join(resultado))

**Exemplos de utilização da tag de negações**

In [30]:
marque_negacao('Thats not ya bitch if she dont love giving you head')

'thats not ya_NEG bitch_NEG if_NEG she_NEG dont_NEG love_NEG giving_NEG you_NEG head_NEG'

In [31]:
marque_negacao('" these hoes like niggas that spend money not talk bout it "')

'" these hoes like niggas that spend money not talk_NEG bout_NEG it_NEG "_NEG'

## **Criando modelos com Pipelines**

* Pipelines são interessantes para reduzir código e automatizar fluxos

In [32]:
from sklearn.pipeline import Pipeline

In [33]:
pipeline_simples = Pipeline([
  ('counts', CountVectorizer()),
  ('classifier', MultinomialNB())
])

* Pipeline que atribui tag de negacoes nas palavras

In [34]:
pipeline_negacoes = Pipeline([
  ('counts', CountVectorizer(tokenizer=lambda text: marque_negacao(text))),
  ('classifier', MultinomialNB())
])

In [35]:
pipeline_simples.fit(tweets,classes)

Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [36]:
pipeline_simples.steps

[('counts',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=None, vocabulary=None)),
 ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

* Gera o modelo de negações

In [37]:
pipeline_negacoes.fit(tweets,classes)

Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        str...      vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

* Etapas do pipeline

In [38]:
pipeline_negacoes.steps

[('counts',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function <lambda> at 0x00000248A1EC9840>,
          vocabulary=None)),
 ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

## Validando os Modelos com Validação Cruzada

* Fazendo o cross validation do modelo

In [39]:
resultados = cross_val_predict(pipeline_simples, tweets, classes, cv=10)

* Medindo a acurácia média do modelo

In [40]:
metrics.accuracy_score(classes,resultados)

0.8599443166686842

* Medidas de validação do modelo

In [41]:
sentimento=['hate_speech','offensive_language','neither']
print (metrics.classification_report(classes,resultados,sentimento))

C:\Users\bruno\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


ValueError: y contains previously unseen labels: [0, 1, 2]

* Matriz de confusão

In [42]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito    0      1     2    All
Real                            
0         51   1264   115   1430
1         49  18894   247  19190
2         17   1779  2367   4163
All      117  21937  2729  24783


## **Modelo com a Tag de Negações**

In [43]:
resultados = cross_val_predict(pipeline_negacoes, tweets, classes, cv=10)

* Medindo a acurácia média do modelo

In [44]:
metrics.accuracy_score(classes,resultados)

0.6276479845055078

In [45]:
sentimento=['hate_speech','offensive_language','neither']
print (metrics.classification_report(classes,resultados,sentimento))

C:\Users\bruno\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


ValueError: y contains previously unseen labels: [0, 1, 2]

* Matriz de confusão

In [ ]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

## ** Avaliando modelo com Bigrams**

In [46]:
'like', 'liked' , 'likes'

('like', 'liked', 'likes')

In [47]:
vectorizer = CountVectorizer(ngram_range=(1,2))
freq_tweets = vectorizer.fit_transform(tweets)
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [48]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [49]:
metrics.accuracy_score(classes,resultados)

0.8753581083807449

In [50]:
sentimento=['hate_speech','offensive_language','neither']
print (metrics.classification_report(classes,resultados,sentimento))

C:\Users\bruno\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


ValueError: y contains previously unseen labels: [0, 1, 2]

In [51]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito    0      1     2    All
Real                            
0        173   1097   160   1430
1        289  18272   629  19190
2         41    873  3249   4163
All      503  20242  4038  24783
